### Keywords count sn remarks

- In this script  we are processing the texts fo SN remarks column and finding the new possible keywords for RCA. We are finf the frequecy of words present in SN_Reamrks and based on frequency come with the keywords.

In [1]:
# Load the file with SN_Remarks data to python
import pandas as pd
keywords = pd.read_excel("C:\\Users\\ijohan\\Desktop\\TNB_PROJECT\\source_data\\may\\data_may_src_listing.xlsx")

In [2]:
#changing the columns name with underscore
keywords.columns = keywords.columns.str.replace(' ','_')

In [3]:
#changing the columns name with underscore
keywords.columns = keywords.columns.str.replace('-','_')

In [4]:
keywords = keywords[['Service_Request_ID','CA_Number','Category', 'Sub_Category','SN_Number','SN_Remarks']]

In [5]:
keywords.loc[:].head()

,Service_Request_ID,CA_Number,Category,Sub_Category,SN_Number,SN_Remarks
0,1000640084,220867052707,OPC Meter Accuracy Test,Low Bill Normal,201000155754,NaN
1,1000655605,210039732001,OPC Meter Accuracy Test,High Bill Normal,201000159237,Internal note 24.05.2019 17:29:23 60068220 P...
2,1000662021,220061331308,Check Meter Reading,Bill Too High,218000610820,Internal note28.05.2019 14:28:19 10095089peng...
3,1000665216,220543677404,Check Meter Reading,Bill Too High,218000612074,Internal note29.05.2019 13:14:06 10092679PENG...
4,1000664889,220715742409,Check Meter Reading,Bill Too High,218000612432,Internal note29.05.2019 15:46:06 10092139peng...


Cleaning SN remarks column, like removing speacial characters and numeric strings from the comment to get clean words and generate unigram

In [6]:
# generate unigrams 
i = keywords.SN_Remarks\
      .str.lower()\
      .str.replace('[^a-z\s]', '')\
      .str.split(expand=True)\
      .stack()

In [7]:
# generate bigrams by concatenating unigram columns
j = i + ' ' + i.shift(-1)
# generate trigrams by concatenating unigram and bigram columns
k = j + ' ' + i.shift(-2)

In [11]:
# concatenate all series vertically, and remove NaNs
k_grams = pd.concat([i, j, k]).dropna().reset_index(drop=True)

In [13]:
# generating starting for 2gram onwards
# concatenate all series vertically, and remove NaNs
k_grams = pd.concat([j, k]).dropna().reset_index(drop=True)

In [15]:
#create column name words
k_grams = pd.DataFrame(k_grams, columns=['words'])

In [17]:
k_grams.head()
k_grams.tail()

,words
1499138,bilbil enbloc iaitu
1499139,enbloc iaitu serentak
1499140,iaitu serentak pada
1499141,serentak pada cukup
1499142,pada cukup bulan


In [18]:
counts = k_grams['words'].value_counts().to_dict()

In [22]:
k_grams_count = pd.DataFrame(counts.items(), columns=['words', 'count'])

In [23]:
k_grams_count.head()

,words,count
0,internal note,15015
1,note pengguna,7681
2,internal note pengguna,7645
3,pengguna maklum,6536
4,note pengguna maklum,4127


In [24]:
len(k_grams_count)

489370

In [25]:
k_grams_count.to_csv("NGRAM_KEYWORD_COUNT_MAY.csv")

### Creating count group by category

In [27]:
# craeting word count based on the category
import pandas as pd
keywords_cat = pd.read_excel("C:\\Users\\ijohan\\Desktop\\TNB_PROJECT\\source_data\\may\\DATA_MAY_UPDATED_WITH24JUNSTATUS_INFO_COMPLAINTS_RCA_SNREMARK_ANALYSIS_24JUNE2019.xlsx")

In [29]:
keywords_cat = keywords_cat[['SN_Number', 'Category', 'SubCategory', 'SN_Remark']]

In [30]:
keywords_cat.Category.unique()

array(['Estimated Bill', 'Wrong Reading', 'Meter Faulty', 'Bill Received',
       'No RMR bill received', 'Meter Crossing'], dtype=object)

In [31]:
# create data frame for each type, breaking main data frame in to smaller based on the Category.
Meter_Crossing = keywords_cat.loc[keywords_cat['Category'] == 'Meter Crossing'] #52
Meter_Faulty = keywords_cat.loc[keywords_cat['Category'] == 'Meter Faulty'] # 863
Wrong_Reading = keywords_cat.loc[keywords_cat['Category'] == 'Wrong Reading'] # 1111
Bill_Received = keywords_cat.loc[keywords_cat['Category'] == 'Bill Received'] # 97
Estimated_Bill = keywords_cat.loc[keywords_cat['Category'] == 'Estimated Bill'] # 109
No_RMR_bill_received = keywords_cat.loc[keywords_cat['Category'] == 'No RMR bill received'] # 9
Product_Change = keywords_cat.loc[keywords_cat['Category'] == 'Product Change'] # 1

# Meter_Crossing, Meter_Faulty, Wrong_Reading, Bill_Received, Estimated_Bill, No_RMR_bill_received 

In [32]:
len(Product_Change)

0

 Take the script of first section of this file which is used for the word freaquency count in SN_remarks column and perform the same action on all the smaller data frame based on the category

In [33]:
# generate unigrams 
i = Meter_Crossing.SN_Remark\
      .str.lower()\
      .str.replace('[^a-z\s]', '')\
      .str.split(expand=True)\
      .stack()

# generate bigrams by concatenating unigram columns
j = i + ' ' + i.shift(-1)
# generate trigrams by concatenating unigram and bigram columns
k = j + ' ' + i.shift(-2)

# concatenate all series vertically, and remove NaNs
Meter_Crossing = pd.concat([i, j, k]).dropna().reset_index(drop=True)
Meter_Crossing = pd.DataFrame(Meter_Crossing, columns=['words'])
counts = Meter_Crossing['words'].value_counts().to_dict()
Meter_Crossing_count = pd.DataFrame(counts.items(), columns=['words', 'count'])
Meter_Crossing_count['category'] = 'Meter Crossing'

In [34]:
# generate unigrams 
i = Wrong_Reading.SN_Remark\
      .str.lower()\
      .str.replace('[^a-z\s]', '')\
      .str.split(expand=True)\
      .stack()

# generate bigrams by concatenating unigram columns
j = i + ' ' + i.shift(-1)
# generate trigrams by concatenating unigram and bigram columns
k = j + ' ' + i.shift(-2)

# concatenate all series vertically, and remove NaNs
Wrong_Reading = pd.concat([i, j, k]).dropna().reset_index(drop=True)
Wrong_Reading = pd.DataFrame(Wrong_Reading, columns=['words'])
counts = Wrong_Reading['words'].value_counts().to_dict()
Wrong_Reading_count = pd.DataFrame(counts.items(), columns=['words', 'count'])
Wrong_Reading_count['category'] = 'Wrong Reading'

In [35]:
#Bill_Received
# generate unigrams 
i = Bill_Received.SN_Remark\
      .str.lower()\
      .str.replace('[^a-z\s]', '')\
      .str.split(expand=True)\
      .stack()

# generate bigrams by concatenating unigram columns
j = i + ' ' + i.shift(-1)
# generate trigrams by concatenating unigram and bigram columns
k = j + ' ' + i.shift(-2)

# concatenate all series vertically, and remove NaNs
Bill_Received = pd.concat([i, j, k]).dropna().reset_index(drop=True)
Bill_Received = pd.DataFrame(Bill_Received, columns=['words'])
counts = Bill_Received['words'].value_counts().to_dict()
Bill_Received_count = pd.DataFrame(counts.items(), columns=['words', 'count'])
Bill_Received_count['category'] = 'Bill Received'

In [36]:
#Estimated_Bill
# generate unigrams 
i = Estimated_Bill.SN_Remark\
      .str.lower()\
      .str.replace('[^a-z\s]', '')\
      .str.split(expand=True)\
      .stack()

# generate bigrams by concatenating unigram columns
j = i + ' ' + i.shift(-1)
# generate trigrams by concatenating unigram and bigram columns
k = j + ' ' + i.shift(-2)

# concatenate all series vertically, and remove NaNs
Estimated_Bill = pd.concat([i, j, k]).dropna().reset_index(drop=True)
Estimated_Bill = pd.DataFrame(Estimated_Bill, columns=['words'])
counts = Estimated_Bill['words'].value_counts().to_dict()
Estimated_Bill_count = pd.DataFrame(counts.items(), columns=['words', 'count'])
Estimated_Bill_count['category'] = 'Estimated Bill'

In [37]:
#No_RMR_bill_received
# generate unigrams 
i = No_RMR_bill_received.SN_Remark\
      .str.lower()\
      .str.replace('[^a-z\s]', '')\
      .str.split(expand=True)\
      .stack()

# generate bigrams by concatenating unigram columns
j = i + ' ' + i.shift(-1)
# generate trigrams by concatenating unigram and bigram columns
k = j + ' ' + i.shift(-2)

# concatenate all series vertically, and remove NaNs
No_RMR_bill_received = pd.concat([i, j, k]).dropna().reset_index(drop=True)
No_RMR_bill_received = pd.DataFrame(No_RMR_bill_received, columns=['words'])
counts = No_RMR_bill_received['words'].value_counts().to_dict()
No_RMR_bill_received_count = pd.DataFrame(counts.items(), columns=['words', 'count'])
No_RMR_bill_received_count['category'] = 'No RMR bill received'

In [38]:
# generate unigrams 
i = Meter_Faulty.SN_Remark\
      .str.lower()\
      .str.replace('[^a-z\s]', '')\
      .str.split(expand=True)\
      .stack()

# generate bigrams by concatenating unigram columns
j = i + ' ' + i.shift(-1)
# generate trigrams by concatenating unigram and bigram columns
k = j + ' ' + i.shift(-2)

# concatenate all series vertically, and remove NaNs
Meter_Faulty = pd.concat([i, j, k]).dropna().reset_index(drop=True)
Meter_Faulty = pd.DataFrame(Meter_Faulty, columns=['words'])
counts = Meter_Faulty['words'].value_counts().to_dict()
Meter_Faulty_count = pd.DataFrame(counts.items(), columns=['words', 'count'])
Meter_Faulty_count['category'] = 'Meter Faulty'

In [39]:
#concat all the data frames # Bill_Received_count , No_RMR_bill_received_count, Product Change
count_category = pd.concat([Meter_Crossing_count, Meter_Faulty_count, Wrong_Reading_count, Estimated_Bill_count,No_RMR_bill_received_count], ignore_index=True)

In [40]:
count_category.head()

,words,count,category
0,meter,113,Meter Crossing
1,pengguna,66,Meter Crossing
2,dan,50,Meter Crossing
3,no,39,Meter Crossing
4,di,38,Meter Crossing


### Tagging based on ethinicity

In [56]:
#SN_Remarks_Working_File_18062019.xlsx
import pandas as pd
ethni = pd.read_excel("C:\\Users\\ijohan\\Desktop\\TNB_PROJECT\\source_data\\may\\SR&COMPLAINTS_ANALYSIS.xlsx")


In [59]:
names_ch = pd.read_csv("C:\\Users\\ijohan\\Desktop\\TNB_PROJECT\\source_data\\may\\names_chinese.csv")
names_malay = pd.read_csv("C:\\Users\\ijohan\\Desktop\\TNB_PROJECT\\source_data\\may\\name_malay.csv")
names_indian = pd.read_csv("C:\\Users\\ijohan\\Desktop\\TNB_PROJECT\\source_data\\may\\name_indian.csv")

In [60]:
ethnicity = ethni[['CA Number','Service Request ID','BP Name']]

In [55]:
ethni.head()

,BP ID,BP Name
0,1200774679,MICHELLE LEE CHIN
1,1102853741,ANDY HO POON SIEW
2,1102592448,SOPHIAN BIN SIAN
3,1200394373,Ahmed Fairuz Bin
4,1100177363,PANG GEOK CHUN


In [62]:
names_ch = names_ch.loc[names_ch['Race'] == 'chinese']
#names_malay = names_malay.loc[names_malay['Race'] == 'Malay']
#names_indian = names_indian.loc[names_indian['Race'] == 'Malay']

In [67]:
#values_ch = [names_ch['Name'].str]
values_ch = names_ch['Name'].tolist()
values_indian = names_indian['name'].tolist()
values_malay = names_malay['name'].tolist()

In [68]:
ethnicity['BP_Name'] = ethnicity['BP Name'].str.lower()

C:\Users\ijohan\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [69]:
ethnicity.head()

,CA Number,Service Request ID,BP Name,BP_Name
0,210036776002,1000670392,MICHELLE LEE CHIN,michelle lee chin
1,220307050002,1000674062,ANDY HO POON SIEW,andy ho poon siew
2,210018089505,1000616785,Ahmed Fairuz Bin,ahmed fairuz bin
3,220191176003,1000620832,PANG GEOK CHUN,pang geok chun
4,220049915905,1000627021,FONG KAR WONG,fong kar wong


In [70]:
# CReating pattern for each names and surnames based on ethinicity

pattern_ch = '|'.join([r'\b{}\b'.format(needle) for needle in values_ch])
pattern_indian = '|'.join([r'\b{}\b'.format(needle) for needle in values_indian])
pattern_malay = '|'.join([r'\b{}\b'.format(needle) for needle in values_malay])

#df = pd.DataFrame(data=data, columns=['id', 'value'])
#result = df[df['value'].str.contains(pattern, regex=True)]

Matching above patterns with each the BP_Name column in the data file.

In [71]:
#chinese
result_ch = ethnicity[ethnicity['BP_Name'].str.contains(pattern_ch, regex=True)]

In [72]:
#INdian
result_indian = ethnicity[ethnicity['BP_Name'].str.contains(pattern_indian, regex=True)]

In [73]:
# Malay
result_malay = ethnicity[ethnicity['BP_Name'].str.contains(pattern_malay, regex=True)]

In [74]:
result_ch['Race'] = 'Chinese'
result_indian['Race'] = 'Indian'
result_malay['Race'] = 'Malay'

C:\Users\ijohan\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
C:\Users\ijohan\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\ijohan\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_ind

In [75]:
result_indian.head()

,CA Number,Service Request ID,BP Name,BP_Name,Race
7,220341154603,1000627274,HARMINDER KAUR A/P,harminder kaur a/p,Indian
21,220002823807,1000637877,SATKUNAVATHI A/P,satkunavathi a/p,Indian
32,220210690102,1000638757,EVALI A/L SUPPIAH,evali a/l suppiah,Indian
33,210041591401,1000639040,NARENDRA KUMAR A/L,narendra kumar a/l,Indian
36,220011569700,1000639421,BHARTI DEVI A/P,bharti devi a/p,Indian


In [76]:
#Merging all data frames
result = pd.concat([result_ch, result_indian,result_malay]).drop_duplicates(subset=['CA Number','Service Request ID','BP Name','Race'], keep=False)

In [77]:
len(result)

15763

In [78]:
# filtering the names where ethinicity tag is not observed based on the name and surnames matching.
ethnicity['Race'] = ''

C:\Users\ijohan\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [79]:
# #substract from the original data and classified based on ethnincity

df = pd.concat([ethnicity,result]).drop_duplicates(keep=False)

In [80]:
df = ethnicity[~ethnicity['Service Request ID'].isin(result['Service Request ID'].values)]

In [81]:
len(df)

3484

In [82]:
df.head()

,CA Number,Service Request ID,BP Name,BP_Name,Race
8,220032066605,1000627421,MENTA CONSTRUCTION,menta construction,
9,220288053304,1000629255,PRIMEWORK (M) SDN,primework (m) sdn,
13,210113162600,1000632215,THE SUBURBAN FOOD,the suburban food,
14,210093934600,1000633750,SOVN (M) SDN BHD,sovn (m) sdn bhd,
18,220078641208,1000636667,RAHIM MASOUDI,rahim masoudi,


In [ ]:
df.to_csv("race_missing.csv")

In [ ]:
result.to_csv("BP_name_with_race.csv")